# Global Food Price Data

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
import tensorflow as tf

In [2]:
df1=pd.read_csv("global_food_prices.csv")
df1.head()

/var/folders/lt/1k4w_6c962s9jjwztgz2qbzr0000gn/T/ipykernel_5840/3278792073.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv("global_food_prices.csv")


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [3]:
#remove unnecessary columns
df2=df1.drop(['adm0_id','adm1_id','mkt_id','cur_name','cm_id','cur_id','pt_id','um_id','mp_commoditysource'],axis='columns')
df2.head()

,adm0_name,adm1_name,mkt_name,cm_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,1,2014,50.0
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,2,2014,50.0
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,3,2014,50.0
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,4,2014,50.0
4,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,5,2014,50.0


In [4]:
#rename the columns
df3=df2.copy()
df3.rename(columns = {'adm0_name':'country', 'adm1_name':'region', 'mkt_name':'city',
                     'cm_name':'commodity','pt_name':'sales_catagory','um_name':'unit',
                     'mp_month':'month','mp_year':'year','mp_price':'price'}, inplace = True)
df3=df3.round(2)
df3

,country,region,city,commodity,sales_catagory,unit,month,year,price
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,1,2014,50.00
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,2,2014,50.00
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,3,2014,50.00
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,4,2014,50.00
4,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,Retail,KG,5,2014,50.00
...,...,...,...,...,...,...,...,...,...
2050633,Zimbabwe,Midlands,Mbilashaba,Beans (sugar) - Retail,Retail,KG,6,2021,233.33
2050634,Zimbabwe,Midlands,Mbilashaba,Toothpaste - Retail,Retail,100 ML,6,2021,112.50
2050635,Zimbabwe,Midlands,Mbilashaba,Laundry soap - Retail,Retail,KG,6,2021,114.00
2050636,Zimbabwe,Midlands,Mbilashaba,Handwash soap - Retail,Retail,250 G,6,2021,59.50


In [5]:
#check the number rows with null values for each column
df3.isnull().sum()

country                0
region            611016
city                   0
commodity              0
sales_catagory         0
unit                   0
month                  0
year                   0
price                  0
dtype: int64

In [6]:
#fill the empty Regions fields with City/Town data
df3['region'].fillna(df3['city'],inplace=True)
df3.drop(['city'],axis='columns',inplace=True)
df3

,country,region,commodity,sales_catagory,unit,month,year,price
0,Afghanistan,Badakhshan,Bread - Retail,Retail,KG,1,2014,50.00
1,Afghanistan,Badakhshan,Bread - Retail,Retail,KG,2,2014,50.00
2,Afghanistan,Badakhshan,Bread - Retail,Retail,KG,3,2014,50.00
3,Afghanistan,Badakhshan,Bread - Retail,Retail,KG,4,2014,50.00
4,Afghanistan,Badakhshan,Bread - Retail,Retail,KG,5,2014,50.00
...,...,...,...,...,...,...,...,...
2050633,Zimbabwe,Midlands,Beans (sugar) - Retail,Retail,KG,6,2021,233.33
2050634,Zimbabwe,Midlands,Toothpaste - Retail,Retail,100 ML,6,2021,112.50
2050635,Zimbabwe,Midlands,Laundry soap - Retail,Retail,KG,6,2021,114.00
2050636,Zimbabwe,Midlands,Handwash soap - Retail,Retail,250 G,6,2021,59.50


In [7]:
#function to remove sales catagory tags from string
def renameCommodity(x):
    name=x.split(' -')
    return name[0]

In [8]:
#apply renameCommodity function on the Commodity column
df4 = df3.copy()
df4['commodity']=df4['commodity'].apply(renameCommodity)
df4.head()

,country,region,commodity,sales_catagory,unit,month,year,price
0,Afghanistan,Badakhshan,Bread,Retail,KG,1,2014,50.0
1,Afghanistan,Badakhshan,Bread,Retail,KG,2,2014,50.0
2,Afghanistan,Badakhshan,Bread,Retail,KG,3,2014,50.0
3,Afghanistan,Badakhshan,Bread,Retail,KG,4,2014,50.0
4,Afghanistan,Badakhshan,Bread,Retail,KG,5,2014,50.0


In [9]:
#remove rows with years less than 2014
df5=df4[df4['year']>=2015]
df5.reset_index()

,index,country,region,commodity,sales_catagory,unit,month,year,price
0,12,Afghanistan,Badakhshan,Bread,Retail,KG,1,2015,50.00
1,13,Afghanistan,Badakhshan,Bread,Retail,KG,2,2015,50.00
2,14,Afghanistan,Badakhshan,Bread,Retail,KG,3,2015,50.00
3,15,Afghanistan,Badakhshan,Bread,Retail,KG,6,2015,50.00
4,16,Afghanistan,Badakhshan,Bread,Retail,KG,7,2015,50.00
...,...,...,...,...,...,...,...,...,...
1441288,2050633,Zimbabwe,Midlands,Beans (sugar),Retail,KG,6,2021,233.33
1441289,2050634,Zimbabwe,Midlands,Toothpaste,Retail,100 ML,6,2021,112.50
1441290,2050635,Zimbabwe,Midlands,Laundry soap,Retail,KG,6,2021,114.00
1441291,2050636,Zimbabwe,Midlands,Handwash soap,Retail,250 G,6,2021,59.50


In [10]:
#check if the regions have enough data
df6=df5.copy()
df6.region=df6.region.apply(lambda x:x.strip())
region_stats=df6.groupby('region')['region'].agg('count').sort_values(ascending=False)
region_stats


region
Yobe               15671
Borno              13965
Homs               13419
Aleppo             12573
South/Amajyepfo    11858
                   ...  
Oumhadjer              6
Amdjarass              3
Granada                2
Leon                   2
Faya                   2
Name: region, Length: 1303, dtype: int64

In [11]:
#remove regions with less than 84 months worth of data
region_stats_less_84=region_stats[region_stats<84]
df6.region=df6.region.apply(lambda x:"" if x in region_stats_less_84 else x)
df7=df6[df6['region']!=""]
df7

,country,region,commodity,sales_catagory,unit,month,year,price
12,Afghanistan,Badakhshan,Bread,Retail,KG,1,2015,50.00
13,Afghanistan,Badakhshan,Bread,Retail,KG,2,2015,50.00
14,Afghanistan,Badakhshan,Bread,Retail,KG,3,2015,50.00
15,Afghanistan,Badakhshan,Bread,Retail,KG,6,2015,50.00
16,Afghanistan,Badakhshan,Bread,Retail,KG,7,2015,50.00
...,...,...,...,...,...,...,...,...
2050633,Zimbabwe,Midlands,Beans (sugar),Retail,KG,6,2021,233.33
2050634,Zimbabwe,Midlands,Toothpaste,Retail,100 ML,6,2021,112.50
2050635,Zimbabwe,Midlands,Laundry soap,Retail,KG,6,2021,114.00
2050636,Zimbabwe,Midlands,Handwash soap,Retail,250 G,6,2021,59.50


In [12]:
country_stats=df7.groupby('country')['country'].agg('count').sort_values(ascending=False)
country_stats

country
Syrian Arab Republic    109104
Bassas da India          96048
Indonesia                71394
Philippines              63052
Burundi                  57286
                         ...  
Moldova Republic of        147
Gabon                      120
Bhutan                     115
Costa Rica                  39
Venezuela                    6
Name: country, Length: 93, dtype: int64

In [13]:
#check if the data of commodities is balanced
commodity_stats=df7.groupby('commodity')['commodity'].agg('count').sort_values(ascending=False)
commodity_stats.describe()
df7


,country,region,commodity,sales_catagory,unit,month,year,price
12,Afghanistan,Badakhshan,Bread,Retail,KG,1,2015,50.00
13,Afghanistan,Badakhshan,Bread,Retail,KG,2,2015,50.00
14,Afghanistan,Badakhshan,Bread,Retail,KG,3,2015,50.00
15,Afghanistan,Badakhshan,Bread,Retail,KG,6,2015,50.00
16,Afghanistan,Badakhshan,Bread,Retail,KG,7,2015,50.00
...,...,...,...,...,...,...,...,...
2050633,Zimbabwe,Midlands,Beans (sugar),Retail,KG,6,2021,233.33
2050634,Zimbabwe,Midlands,Toothpaste,Retail,100 ML,6,2021,112.50
2050635,Zimbabwe,Midlands,Laundry soap,Retail,KG,6,2021,114.00
2050636,Zimbabwe,Midlands,Handwash soap,Retail,250 G,6,2021,59.50


In [14]:
df8=df7.copy()
commodity_stats_less=commodity_stats[commodity_stats<2000]
df8.commodity=df8.commodity.apply(lambda x:"" if x in commodity_stats_less else x)
df8=df8[df8['commodity']!=""]
df8


,country,region,commodity,sales_catagory,unit,month,year,price
12,Afghanistan,Badakhshan,Bread,Retail,KG,1,2015,50.00
13,Afghanistan,Badakhshan,Bread,Retail,KG,2,2015,50.00
14,Afghanistan,Badakhshan,Bread,Retail,KG,3,2015,50.00
15,Afghanistan,Badakhshan,Bread,Retail,KG,6,2015,50.00
16,Afghanistan,Badakhshan,Bread,Retail,KG,7,2015,50.00
...,...,...,...,...,...,...,...,...
2050631,Zimbabwe,Midlands,Sugar,Retail,KG,6,2021,118.38
2050632,Zimbabwe,Midlands,Salt,Retail,KG,6,2021,71.00
2050634,Zimbabwe,Midlands,Toothpaste,Retail,100 ML,6,2021,112.50
2050635,Zimbabwe,Midlands,Laundry soap,Retail,KG,6,2021,114.00


In [15]:
#mean price of each commodity per year for each region
df9=df8.groupby(['country','commodity','sales_catagory','year','unit'],as_index=False)['price'].mean().round(2)
df9.head(50)

,country,commodity,sales_catagory,year,unit,price
0,Afghanistan,Bread,Retail,2015,KG,44.39
1,Afghanistan,Bread,Retail,2016,KG,44.70
2,Afghanistan,Bread,Retail,2017,KG,44.45
3,Afghanistan,Bread,Retail,2018,KG,44.29
4,Afghanistan,Bread,Retail,2019,KG,44.51
5,Afghanistan,Bread,Retail,2020,KG,50.88
6,Afghanistan,Bread,Retail,2021,KG,52.53
7,Afghanistan,Exchange rate,Retail,2015,USD/LCU,61.24
8,Afghanistan,Exchange rate,Retail,2016,USD/LCU,67.70
9,Afghanistan,Exchange rate,Retail,2017,USD/LCU,68.02


In [71]:
def remove_price_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby(['country','commodity'],as_index=False):
        m = np.mean(subdf.price)
        st=np.std(subdf.price)
        reduced_df=subdf[(subdf.price>(m-(st*2)))&(subdf.price<=(m+(st/2)))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df10=remove_price_outliers(df9)
df10



,country,commodity,sales_catagory,year,unit,price
0,Afghanistan,Bread,Retail,2015,KG,44.39
1,Afghanistan,Bread,Retail,2016,KG,44.70
2,Afghanistan,Bread,Retail,2017,KG,44.45
3,Afghanistan,Bread,Retail,2018,KG,44.29
4,Afghanistan,Bread,Retail,2019,KG,44.51
...,...,...,...,...,...,...
5266,Zimbabwe,Sorghum,Retail,2019,KG,0.34
5267,Zimbabwe,Sugar,Retail,2021,KG,118.20
5268,Zimbabwe,Toothpaste,Retail,2020,100 ML,109.08
5269,Zimbabwe,Wheat,Retail,2015,KG,0.48


In [17]:
def hot_encode(df,column):
    df = df.copy()
    dummies = pd.get_dummies(df[column],prefix=column)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop(column,axis=1)
    return df

In [19]:
dummies = pd.get_dummies(df10['country'])
def encode_col(df):
    encoded=df10.copy()
    for column in ['commodity', 'sales_catagory', 'unit']:
        encoded=hot_encode(encoded,column)
    return encoded
df10=encode_col(df10)
df10=pd.concat([df10,dummies],axis="columns")
df10=df10.drop(['country'],axis=1)
df10


,year,price,commodity_Apples,commodity_Apples (red),commodity_Bananas,commodity_Beans,commodity_Beans (dry),commodity_Beans (niebe),commodity_Beans (red),commodity_Beans (white),...,Thailand,Timor-Leste,Togo,Turkey,Uganda,Ukraine,United Republic of Tanzania,Yemen,Zambia,Zimbabwe
0,2015,44.39,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016,44.70,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,44.45,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,44.29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,44.51,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,2019,0.34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5267,2021,118.20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5268,2020,109.08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5269,2015,0.48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
y=df10.price
X = df10.drop('price',axis="columns")

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.2,shuffle=True,random_state=9)


In [36]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

lr_clf=DecisionTreeRegressor()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)


0.8687743774669564

In [30]:
X.columns

Index(['year', 'commodity_Apples', 'commodity_Apples (red)',
       'commodity_Bananas', 'commodity_Beans', 'commodity_Beans (dry)',
       'commodity_Beans (niebe)', 'commodity_Beans (red)',
       'commodity_Beans (white)', 'commodity_Bread',
       ...
       'Thailand', 'Timor-Leste', 'Togo', 'Turkey', 'Uganda', 'Ukraine',
       'United Republic of Tanzania', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', length=349)

In [119]:
np.where(X.columns=='commodity_Oil')[0][0]

95

In [124]:
def predict_price(location,commodity,sales_catagory):
    loc_index=np.where(X.columns==location)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = commodity
    x[1] = sales_catagory
    if loc_index >=0:
        x[loc_index] = 1
    return lr_clf.predict([x])[0]

In [126]:
import pickle
with open('food_price_model.pickle','wb')as f:
    pickle.dump(lr_clf,f)

In [128]:
import json
columns = {
    'data_columns':[col.lower() for col in X.columns]
}
with open("columns.json","w")as f:
    f.write(json.dumps(columns))